In [1]:
import pathlib
import numpy as np
from pivpy import io, pivpy, graphics

In [2]:
fieldsDir = pathlib.Path('../../pivpy/data/openpiv_txt/Gamma1_Gamma2_tutorial_notebook')

In [3]:
# Global variables
n = 4

In [4]:
dataBaseXarray = io.load_directory(
    path = fieldsDir,
    basename = "OpenPIVtxtFilePair?",
    ext = ".txt"
)

found 10 files


In [5]:
dataBaseXarray.piv.Γ1(n, convCoords=True)

C:\Users\nepomnyi\Box\OpenPIVdevelopment\pivpy\pivpy\compute_funcs.py:61: RuntimeWarning: invalid value encountered in divide
  Γ1 = np.nanmean(np.divide(np.subtract(np.multiply(PMx,v), np.multiply(PMy,u)), np.multiply(np.sqrt(np.add(np.square(PMx), np.square(PMy))), np.sqrt(np.add(np.square(u), np.square(v))))))
C:\Users\nepomnyi\Box\OpenPIVdevelopment\pivpy\pivpy\compute_funcs.py:61: RuntimeWarning: Mean of empty slice
  Γ1 = np.nanmean(np.divide(np.subtract(np.multiply(PMx,v), np.multiply(PMy,u)), np.multiply(np.sqrt(np.add(np.square(PMx), np.square(PMy))), np.sqrt(np.add(np.square(u), np.square(v))))))


<xarray.Dataset>
Dimensions:       (x: 79, y: 63, t: 10)
Coordinates:
  * x             (x) float64 0.9412 1.882 2.824 3.765 ... 72.47 73.41 74.35
  * y             (y) float64 59.29 58.35 57.41 56.47 ... 2.824 1.882 0.9412
  * t             (t) int32 0 1 2 3 4 5 6 7 8 9
Data variables:
    u             (x, y, t) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    v             (x, y, t) float64 -0.0 -0.0 -0.0 -0.0 ... -0.0 -0.0 -0.0 -0.0
    chc           (x, y, t) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    xCoordinates  (x, y, t) float64 0.9412 0.9412 0.9412 ... 74.35 74.35 74.35
    yCoordinates  (x, y, t) float64 59.29 59.29 59.29 ... 0.9412 0.9412 0.9412
    Γ1            (x, y, t) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Attributes:
    delta_t:  0.0
    files:    [WindowsPath('../../pivpy/data/openpiv_txt/Gamma1_Gamma2_tutori...

In [ ]:
dataBaseXarray.piv.Γ2(n, convCoords=False) # False, because the coordinates have already been converted
                                           # at the previous line

In [ ]:
# Plot Γ1 fields.
for i in range(dataBaseXarray.coords['t'].values.size):
    levelsΓ1 = np.linspace(np.min(field['Γ1'].isel(t=i).values), np.max(field['Γ1'].isel(t=i).values), 1000)   
        figΓ1, axΓ1 = plt.subplots(nrows=1, ncols=1, clear=True, num="figGamma1", figsize=(15,10))
        field['Γ1'].isel(t=i).plot.contourf(x="x",
                                            y="y",
                                            levels=levelsΓ1,
                                            cmap=plt.get_cmap("RdYlBu"),
                                            ax=axΓ1
        )
        # Save the plots.
        # # Resize the figure to the full screen in order to save it as 
        # # screen sized image.
        # plt.get_current_fig_manager().full_screen_toggle()      
        # # The stupid full_screen_toggle works only if I show the picture first, but I don't 
        # # want to show it. That's why I use the closing operation.
        # figΓ1.show()
        nameΓ1 = saveToDir / 'Gamma1AveragedOver{}correlationsBatch{}.png'.format(N,i)
        figΓ1.savefig(str(nameΓ1))
        plt.close(fig=figΓ1)

        plt.close('all')